In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.text import *
from fastai.imports import *


import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torch.utils.data import DataLoader

import pandas as pd

In [3]:
from lang_model import *

In [4]:
!conda list fastai

/Users/rchaks/.zshenv:1: bad assignment
# packages in environment at /Users/rchaks/opt/anaconda3/envs/aml-proj:
#
# Name                    Version                   Build  Channel
fastai                    1.0.61                   pypi_0    pypi


In [5]:
path = Path('../data/')
path

PosixPath('../data')

In [6]:
from sklearn.model_selection import train_test_split

def load_data(df: pd.DataFrame):
    train_sentences, valid_sentences = train_test_split(sentences, test_size= 0.1, random_state = 121)

    data_lm = TextLMDataBunch.from_df(path = path, train_df= sentences, valid_df= valid_sentences, text_cols= "0")
    return data_lm

# Read CSV

In [7]:
sentences = pd.read_csv("../data/lyrics_preprocessed.csv")
sentences

,0
0,"Your man on the road, he doin promo"
1,"You said, ""Keep our business on the low-low"""
2,Im just tryna get you out the friend zone
3,Cause you look even better than the photos
4,"I cant find your house, send me the info"
...,...
40819,Scene goes frame by frame
40820,Who swayed to suede
40821,Youll hear it all dont call
40822,Throw away everything Ive written you oh


In [8]:
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vectorizer = TfidfVectorizer(decode_error='ignore'
#                             # ,ngram_range = (1,2)
#                             ,use_idf = True
#                             ,strip_accents = 'ascii'
#                             # ,smooth_idf = False
#                             )
# song_vectors = vectorizer.fit_transform(sentences["0"])
# print(song_vectors.shape)
# # 7042 -> Training instances
# # 21954 -> Total words
# print(min(vectorizer.idf_), max(vectorizer.idf_))
# idf_vocab = pd.Series(vectorizer.idf_, index= vectorizer.get_feature_names())

# # Selecting top 10% most frequently occouring words
# # Selecting top 10% most frequently occouring words
# vocab = idf_vocab[idf_vocab <= idf_vocab.quantile(q = 0.15)]
# print(idf_vocab.quantile(q = 0.2), len(vocab))

# vocab = pd.Series([t for t in vocab.index if not t.isnumeric()], dtype=str)
# vocab


In [9]:
# vectorizer = CountVectorizer(decode_error='ignore'
#                             # ,ngram_range = (1,2)
#                             ,strip_accents = 'ascii'
#                             # ,smooth_idf = False
#                             ,vocabulary = set(vocab)
#                         )
# X = vectorizer.fit_transform(sentences["0"])
# print(X.shape)
# # 7042 -> Training instances
# # 21954 -> Total words
# vocab= vectorizer.get_feature_names_out()
# print(vocab)
# print(X.toarray()[1])

In [10]:
data_lm = load_data(sentences)

/Users/rchaks/opt/anaconda3/envs/aml-proj/lib/python3.9/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


# f

In [11]:
# !conda list spacy

In [12]:
trn_dl = data_lm.train_dl
val_dl = data_lm.valid_dl

In [13]:
#export
def bn_drop_lin(n_in, n_out, bn=True, initrange=0.01,p=0, bias=True, actn=nn.LeakyReLU(inplace=True)):
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    linear = nn.Linear(n_in, n_out, bias=bias)
    if initrange:linear.weight.data.uniform_(-initrange, initrange)
    if bias: linear.bias.data.zero_()
    layers.append(linear)
    if actn is not None: layers.append(actn)
    return layers

In [14]:
#export
def bn_drop_lin(n_in, n_out, bn=True, initrange=0.01,p=0, bias=True, actn=nn.LeakyReLU(inplace=True)):
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    linear = nn.Linear(n_in, n_out, bias=bias)
    if initrange:linear.weight.data.uniform_(-initrange, initrange)
    if bias: linear.bias.data.zero_()
    layers.append(linear)
    if actn is not None: layers.append(actn)
    return layers

In [15]:
from fastai.text.learner import language_model_learner
from fastai.text.models import TransformerXL
learn = language_model_learner(data_lm, arch=TransformerXL)
# learn.load('lyrics_fine_tuned_novel')

/Users/rchaks/opt/anaconda3/envs/aml-proj/lib/python3.9/site-packages/fastai/text/learner.py:215: UserWarning: There are no pretrained weights for that architecture yet!
  warn("There are no pretrained weights for that architecture yet!")


In [16]:
encoder = deepcopy(learn.model[0])
encoder

TransformerXL(
  (encoder): Embedding(5264, 410)
  (pos_enc): PositionalEncoding()
  (drop_emb): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0): DecoderLayer(
      (mhra): MultiHeadRelativeAttention(
        (attention): Linear(in_features=410, out_features=1230, bias=False)
        (out): Linear(in_features=410, out_features=410, bias=False)
        (drop_att): Dropout(p=0.1, inplace=False)
        (drop_res): Dropout(p=0.1, inplace=False)
        (ln): LayerNorm((410,), eps=1e-05, elementwise_affine=True)
        (r_attn): Linear(in_features=410, out_features=410, bias=False)
      )
      (ff): SequentialEx(
        (layers): ModuleList(
          (0): Linear(in_features=410, out_features=2100, bias=True)
          (1): ReLU(inplace=True)
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=2100, out_features=410, bias=True)
          (4): Dropout(p=0.1, inplace=False)
          (5): MergeLayer()
          (6): LayerNorm((410,), eps=1e-05, e

In [17]:
x, y = next(iter(trn_dl))
x.size(), y.size()

(torch.Size([64, 70]), torch.Size([64, 70]))

In [18]:
outs = encoder(x)

/Users/rchaks/opt/anaconda3/envs/aml-proj/lib/python3.9/site-packages/fastai/text/models/transformer.py:114: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1634272478997/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1273.)
  attn_score = attn_score.float().masked_fill(mask, -float('inf')).type_as(attn_score)


In [19]:
outs[-1][-1].size()

torch.Size([64, 70, 410])

In [20]:
[out.size() for out in outs[-1]]

[torch.Size([64, 70, 410])]

In [21]:
generator = deepcopy(learn.model) 

In [22]:
generator.load_state_dict(learn.model.state_dict())

<All keys matched successfully>

In [23]:
#export
class TextDicriminator(nn.Module):
    def __init__(self,encoder, nh, bn_final=True):
        super().__init__()
        #encoder
        self.encoder = encoder
        #classifier
        layers = []
        layers+=bn_drop_lin(nh*3,nh,bias=False)
        layers += bn_drop_lin(nh,nh,p=0.25)
        layers+=bn_drop_lin(nh,1,p=0.15,actn=nn.Sigmoid())
        if bn_final: layers += [nn.BatchNorm1d(1)]
        self.layers = nn.Sequential(*layers)
    
    def pool(self, x, bs, is_max):
        f = F.adaptive_max_pool1d if is_max else F.adaptive_avg_pool1d
        return f(x.permute(0,2,1), (1,)).view(bs,-1)
    
    def forward(self, inp,y=None):
        raw_outputs, outputs = self.encoder(inp)
        output = outputs[-1]
        bs,sl,_ = output.size()
        avgpool = self.pool(output, bs, False)
        mxpool = self.pool(output, bs, True)
        x = torch.cat([output[:,-1], mxpool, avgpool], 1)
        out = self.layers(x)
        return out

In [24]:
disc = TextDicriminator(encoder,400)#.cuda()

In [25]:
optimizerD = optim.Adam(disc.parameters(), lr = 3e-4)
optimizerG = optim.Adam(generator.parameters(), lr = 3e-3, betas=(0.7, 0.8))

In [26]:
#export
def seq_gumbel_softmax(input):
    samples = []
    bs,sl,nc = input.size()
    for i in range(sl): 
        z = F.gumbel_softmax(input[:,i,:])
        samples.append(torch.multinomial(z,1))
    samples = torch.stack(samples).transpose(1,0).squeeze(2) 
    return samples

In [27]:
#export
from tqdm import tqdm

In [28]:
#export
def reinforce_loss(input,sample,reward):
    loss=0
    bs,sl = sample.size()
    for i in range(sl):
        loss += -input[:,i,sample[:,i]] * reward
    return loss/sl

In [29]:
#export
def step_gen(ds,gen,disc,optG,crit=None):
    gen.train(); disc.train()
    x,y = ds
    bs,sl = x.size()
    fake,_,_ = gen(x)
    gen.zero_grad()
    fake_sample =seq_gumbel_softmax(fake)
    with torch.no_grad():
        gen_loss = reward = disc(fake_sample)
        if crit: gen_loss = crit(fake,fake_sample,reward.squeeze(1))
        gen_loss = gen_loss.mean()
    gen_loss.requires_grad_(True)
    gen_loss.backward()
    optG.step()
    return gen_loss.data.item()

In [30]:
#export
def step_disc(ds,gen,disc,optD,d_iters):
    for j in range(d_iters):
        gen.eval(); disc.train()
        with torch.no_grad():
            fake,_,_ = gen(x)
            fake_sample = seq_gumbel_softmax(fake)
        disc.zero_grad()
        fake_loss = disc(fake_sample)
        real_loss = disc(y.view(bs,sl))
        disc_loss = (fake_loss-real_loss).mean(0)
        disc_loss.backward()
        optimizerD.step()
    return disc_loss.data.item()

In [31]:
#export
def evaluate(ds,gen,disc,crit=None):
    with torch.no_grad():
        x, y = ds
        bs,sl = x.size()
        fake,_,_ = gen(x)
        fake_sample =seq_gumbel_softmax(fake)
        gen_loss = reward = disc(fake_sample)
        if crit: gen_loss = crit(fake,fake_sample,reward.squeeze(1))
        gen_loss = gen_loss.mean()
        fake_sample = seq_gumbel_softmax(fake)
        fake_loss = disc(fake_sample).mean(0).view(1).data.item()
        real_loss = disc(y.view(bs,sl)).mean(0).view(1).data.item()
        disc_loss = (fake_loss-real_loss).mean(0).view(1).data.item()
    return fake,gen_loss,disc_loss,fake_loss

In [32]:
#export
def train(gen, disc, epochs, trn_dl, val_dl, optimizerD, optimizerG, crit=None,first=True):
    gen_iterations = 0
    
    for epoch in range(epochs):
        
        gen.train(); disc.train()
        n = len(trn_dl)
        #train loop
        with tqdm(total=n) as pbar:
            for i, ds in enumerate(trn_dl):
                gen_loss = step_gen(ds,gen,disc,optimizerG,crit)
                gen_iterations += 1
                d_iters = 3
                disc_loss = step_disc(ds,gen,disc,optimizerD,d_iters)
                pbar.update()
        print(f'Epoch {epoch}:')
        print('Train Loss:')
        print(f'Loss_D {disc_loss}; Loss_G {gen_loss} Ppx {torch.exp(lm_loss(fake,y))}')
        print(f'D_real {real_loss}; Loss_D_fake {fake_loss}')
        
        disc.eval(), gen.eval()
        with tqdm(total=len(val_dl)) as pbar:
            for i, ds in enumerate(val_dl):
                fake,gen_loss,disc_loss,fake_loss = evaluate(ds,gen,disc,crit)
                pbar.update()
        print('Valid Loss:')
        print(f'Loss_D {disc_loss}; Loss_G {gen_loss} Ppx {torch.exp(lm_loss(fake,ds[-1]))}')
        print(f'D_real {real_loss}; Loss_D_fake {fake_loss}')

In [ ]:
#export
nh = {'AWD':400,'XL':410}
crits={'gumbel':None,'reinforce':reinforce_loss}

    learn = language_model_learner(data_lm, arch=TransformerXL)
    learn.load(pretrained)
    encoder = deepcopy(learn.model[0])
    
    generator = deepcopy(learn.model)
    generator.load_state_dict(learn.model.state_dict())
    disc = TextDicriminator(encoder,nh[model]).cuda()
    
    disc.train()
    generator.train()
    
    #create optimizers
    optimizerD = optim.Adam(disc.parameters(), lr = 3e-4)
    optimizerG = optim.Adam(generator.parameters(), lr = 3e-3, betas=(0.7, 0.8))
    
    print(f'training for {epochs} epochs')
    train(generator, disc, epochs, trn_dl, val_dl, optimizerD, optimizerG, first=False)
    
    #save model
    learn.model.load_state_dict(generator.state_dict())
    print(f'saving model to {path}/{filename}_{model}_gan_{crit}')
    learn.save(filename+'_'+model+'_gan_'+crit)
    
    #generate output from validation set
    if preds:
        print(f'generating predictions and saving to {path}/{filename}_{model}_preds.txt;')
        get_valid_preds(learn,data_lm,filename+'_'+model+'_preds.txt')

In [33]:
# #export
# nh = {'AWD':400,'XL':410}
# crits={'gumbel':None,'reinforce':reinforce_loss}

# #train a language model with gan objective
# def run(path,filename,pretrained,model,crit=None,preds=True,epochs=6):
#     #load data after running preprocess
#     print(f'loading data from {path}/{filename};')
#     data_lm = load_data(path, filename)
#     trn_dl = data_lm.train_dl
#     val_dl = data_lm.valid_dl
    
#     #select encoder for model
#     print(f'training text gan model {model}; pretrained from {pretrained};')
#     learn = language_model_learner(data_lm, arch=models[model])
#     learn.load(pretrained)
#     encoder = deepcopy(learn.model[0])
    
#     generator = deepcopy(learn.model)
#     generator.load_state_dict(learn.model.state_dict())
#     disc = TextDicriminator(encoder,nh[model]).cuda()
    
#     disc.train()
#     generator.train()
    
#     #create optimizers
#     optimizerD = optim.Adam(disc.parameters(), lr = 3e-4)
#     optimizerG = optim.Adam(generator.parameters(), lr = 3e-3, betas=(0.7, 0.8))
    
#     print(f'training for {epochs} epochs')
#     train(generator, disc, epochs, trn_dl, val_dl, optimizerD, optimizerG, first=False)
    
#     #save model
#     learn.model.load_state_dict(generator.state_dict())
#     print(f'saving model to {path}/{filename}_{model}_gan_{crit}')
#     learn.save(filename+'_'+model+'_gan_'+crit)
    
#     #generate output from validation set
#     if preds:
#         print(f'generating predictions and saving to {path}/{filename}_{model}_preds.txt;')
#         get_valid_preds(learn,data_lm,filename+'_'+model+'_preds.txt')

In [34]:
#export
if __name__ == '__main__': fire.Fire(run)

ERROR: The function received no value for the required argument: path
Usage: ipykernel_launcher.py PATH FILENAME PRETRAINED MODEL <flags>
  optional flags:        --crit | --preds | --epochs

For detailed information on this command, run:
  ipykernel_launcher.py --help


FireExit: 2

/Users/rchaks/opt/anaconda3/envs/aml-proj/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# !notebook2script.py textgan.ipynb

In [36]:
!jupyter nbconvert --to script textgan.ipynb

/Users/rchaks/.zshenv:1: bad assignment
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbconvert` not found.
